In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
arima_models = {}
arima_evals = {}
for crypto in splitted_data.keys():
    crypto_models = {}
    crypto_evals = {}
    for i in range(len(scenarios)):
        train_df = splitted_data[crypto]['trains'][i]
        test_df = splitted_data[crypto]['tests'][i]
        
        X_train, y_train = prepare_ml_data(train_df)
        X_test, y_test = prepare_ml_data(test_df)

        arima_model = auto_arima(y_train, seasonal=True, stepwise=True, trace=True)
        all_predictions = arima_model.predict(n_periods=len(test_df) + 90 * 24)
        predictions = all_predictions[:len(test_df)]
        f90_prices = all_predictions[len(test_df):]
        crypto_evals[scenarios[i]] = eval(y_test, predictions)
        crypto_models[scenarios[i]] = arima_model
        forecast_plot('ARIMA', y_train, predictions, f90_prices, crypto,scenarios[i], save=True)
    arima_models[crypto] = crypto_models
    arima_evals[crypto] = crypto_evals
display(pd.DataFrame(arima_evals))
pd.DataFrame(arima_evals).to_csv('ARIMA_evals.csv')